# Import the models

In [1]:
from qdrant_client import models
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

In [2]:
# Sentence Transformers chứa nhiều mô hình nhúng (embedding)
# Mô hình "all-MiniLM-L6-v2" đáp ứng cơ bản về chất lượng và và tốc độ
encoder = SentenceTransformer("..\\embedding_models\\all-MiniLM-L6-v2\\")

# Add the dataset

In [5]:
# "all-MiniLM-L6-v2" sẽ mã hóa dữ liệu được cung cấp
# Dưới đây là một ví dữ liệu sẽ được encoding
documents = [
    {
        "name": "lịch sử việt nam thời kì pháp thuộcthuộc",
        "description": "đây là một đoạn văn tiếng việt được thay đổi để thử nghiệm liệu mô hình embedding có thể embed văn bản tiếng việt hay không?",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game",
        "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Brave New World",
        "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Dune",
        "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Foundation",
        "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "The War of the Worlds",
        "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "The Hunger Games",
        "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "The Andromeda Strain",
        "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "The Three-Body Problem",
        "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]

# Define storage location

In [6]:
client = QdrantClient(url="http://localhost:6333")

# Create a collection

In [7]:
client.create_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)
# Tham số vector_size xác định kích thước của các vector cho một collection cụ thể
# Nếu kích thước giũa chúng khác nhau -> không thể tính toán khoảng cách giữa chúng
# Có thể sử dụng model.get_sentence_embedding_dimension() để lấy chiều của mô hình đang sử dụng
# Tham số distance cho phép chỉ định hàm được sử dụng để đo khoảng cách giữa 2 vector

True

# Upload data to collection

In [8]:
client.upload_points(
    collection_name="my_books",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(documents)
    ],
)

# Ask the engine a question

In [9]:
# Bây giờ, dữ liệu đã được lưu trữ trong Qdrant. Chúng ta có thể hỏi những câu hỏi
# và nhận được kết quả có liên quan về mặt ngữ nghĩa
hits = client.query_points(
    collection_name="my_books",
    query=encoder.encode("pháp thuoc").tolist(),
    limit=3,
).points

for hit in hits:
    print(hit.payload, f"score: {hit.score}")

{'name': 'lịch sử việt nam thời kì pháp thuộcthuộc', 'description': 'đây là một đoạn văn tiếng việt được thay đổi để thử nghiệm liệu mô hình embedding có thể embed văn bản tiếng việt hay không?', 'author': 'H.G. Wells', 'year': 1895} score: 0.20432398
{'name': 'Snow Crash', 'description': 'A futuristic world where the internet has evolved into a virtual reality metaverse.', 'author': 'Neal Stephenson', 'year': 1992} score: 0.14487001
{'name': "The Hitchhiker's Guide to the Galaxy", 'description': 'A comedic science fiction series following the misadventures of an unwitting human and his alien friend.', 'author': 'Douglas Adams', 'year': 1979} score: 0.110646285


# Narrow down the query
Thu hẹp truy vấn

In [13]:
# Những cuốn sách mới nhất xuất bản vào những năm 2000s thì sao?
hits = client.query_points(
    collection_name="my_books",
    query=encoder.encode("doan").tolist(),
    query_filter=models.Filter(
        must=[models.FieldCondition(key="year", range=models.Range(gte=1800))]
    ),
    limit=1,
).points

for hit in hits:
    print(hit.payload, f"score: {hit.score}")

{'name': 'lịch sử việt nam thời kì pháp thuộcthuộc', 'description': 'đây là một đoạn văn tiếng việt được thay đổi để thử nghiệm liệu mô hình embedding có thể embed văn bản tiếng việt hay không?', 'author': 'H.G. Wells', 'year': 1895} score: 0.21399024
